## Parameters

In [109]:
input_data='./input_data/'
output_data='./output_data/'

## Libraries

In [141]:
import numpy as np
import pandas as pd
import datetime

Load the worst case impact table. 

WC1 and WC2 are the two worst case impact predictions giving reduction in flow by month for 2070.
- *WC1*: Hamududu & Killingtviet 2016
- *WC2*: Harrison & Whittington, 2002

In [120]:
cc_cases=pd.read_csv(input_data + 'climate_change_worst_cases.csv').set_index('Month')
cc_cases

,WC1,WC2
Month,,
10,-0.16,-0.243920
11,-0.20,-0.296011
12,-0.17,-0.396367
1,-0.18,-0.326546
2,-0.16,-0.324661
3,-0.41,-0.395765
4,-0.29,-0.386358
5,-0.24,-0.362866
6,-0.24,-0.350233


In [142]:
daily_flow=pd.read_csv(output_data + 'ngonye_flow_daily.csv')

daily_flow=daily_flow.set_index(pd.DatetimeIndex(daily_flow['Date']))
daily_flow=daily_flow.drop(['LaggedDate','VicFalls','Conversion','Flow_difference','Volume','MonthId','Date','WaterMonth','WaterDay','WaterWeek','Exceedance'],axis=1)
daily_flow.head(4)

,Flow,Year,Month,Day,WaterYear
Date,,,,,
1924-10-01,111.133124,1924,10,1,1924
1924-10-02,111.133124,1924,10,2,1924
1924-10-03,111.133124,1924,10,3,1924
1924-10-04,111.133124,1924,10,4,1924


In [143]:
sims=pd.DataFrame(np.arange(1,61,1),columns=['SimNo'])
sims['StartDate']=pd.date_range(start=datetime.date(1933, 10, 1), periods=60, freq=pd.tseries.offsets.DateOffset(years=1))
sims

,SimNo,StartDate
0,1,1933-10-01
1,2,1934-10-01
2,3,1935-10-01
3,4,1936-10-01
4,5,1937-10-01
5,6,1938-10-01
6,7,1939-10-01
7,8,1940-10-01
8,9,1941-10-01
9,10,1942-10-01


Calculate the Climate Change Rate which is 0% in 1990 and rises to 100% in 2070. This sets the proportion of the overall climate change impact (by 2070) which is assumed to apply to any given date.

In [151]:
daily_flow['CCRate']=daily_flow.apply(lambda x: (0 if x['Year']<1991 else (x['Year']-1990)/80),axis=1)
daily_flow['WC1_Monthly']=daily_flow.join(cc_cases,on='Month')['WC1']
daily_flow['WC2_Monthly']=daily_flow.join(cc_cases,on='Month')['WC2']
daily_flow['WC1_Adjusted']=daily_flow['Flow']*(1+daily_flow['CCRate']*(-daily_flow['WC1_Monthly']))
daily_flow['WC2_Adjusted']=daily_flow['Flow']*(1+daily_flow['CCRate']*(-daily_flow['WC2_Monthly']))
daily_flow

,Flow,Year,Month,Day,WaterYear,CCRate,WC1_Monthly,WC2_Monthly,WC1_Adjusted,WC2_Adjusted
Date,,,,,,,,,,
1924-10-01,111.133124,1924,10,1,1924,0.0000,-0.16,-0.243920,111.133124,111.133124
1924-10-02,111.133124,1924,10,2,1924,0.0000,-0.16,-0.243920,111.133124,111.133124
1924-10-03,111.133124,1924,10,3,1924,0.0000,-0.16,-0.243920,111.133124,111.133124
1924-10-04,111.133124,1924,10,4,1924,0.0000,-0.16,-0.243920,111.133124,111.133124
1924-10-05,111.133124,1924,10,5,1924,0.0000,-0.16,-0.243920,111.133124,111.133124
...,...,...,...,...,...,...,...,...,...,...
2017-09-26,203.017496,2017,9,26,2016,0.3375,-0.14,-0.293764,212.610072,223.145758
2017-09-27,203.017496,2017,9,27,2016,0.3375,-0.14,-0.293764,212.610072,223.145758
2017-09-28,203.017496,2017,9,28,2016,0.3375,-0.14,-0.293764,212.610072,223.145758


In [145]:
tmp=daily_flow.reset_index().head(4018)[['Date','Flow']].copy()
tmp['Date']=pd.date_range(start=datetime.date(2017, 10, 1), periods=4018, freq='D')
tmp=tmp.set_index('Date')
tmp['Day']=tmp.index.day
tmp['Month']=tmp.index.month
tmp['Year']=tmp.index.year
tmp['WaterYear']=tmp.apply((lambda x: (x['Year'] if x['Month']>=10 else x['Year']-1)),axis=1)
tmp['CCRate']=0
tmp


,Flow,Day,Month,Year,WaterYear,CCRate
Date,,,,,,
2017-10-01,111.133124,1,10,2017,2017.0,0
2017-10-02,111.133124,2,10,2017,2017.0,0
2017-10-03,111.133124,3,10,2017,2017.0,0
2017-10-04,111.133124,4,10,2017,2017.0,0
2017-10-05,111.133124,5,10,2017,2017.0,0
...,...,...,...,...,...,...
2028-09-26,259.369380,26,9,2028,2027.0,0
2028-09-27,259.369380,27,9,2028,2027.0,0
2028-09-28,254.145972,28,9,2028,2027.0,0


In [146]:
daily=daily_flow.append(tmp)
daily

,Flow,Year,Month,Day,WaterYear,CCRate
Date,,,,,,
1924-10-01,111.133124,1924,10,1,1924.0,0.0
1924-10-02,111.133124,1924,10,2,1924.0,0.0
1924-10-03,111.133124,1924,10,3,1924.0,0.0
1924-10-04,111.133124,1924,10,4,1924.0,0.0
1924-10-05,111.133124,1924,10,5,1924.0,0.0
...,...,...,...,...,...,...
2028-09-26,259.369380,2028,9,26,2027.0,0.0
2028-09-27,259.369380,2028,9,27,2027.0,0.0
2028-09-28,254.145972,2028,9,28,2027.0,0.0


In [116]:
daily_future=pd.DataFrame(index=pd.date_range(start='10/1/2017', end='09/30/2053', freq='D'))
daily_future['Year']=daily_future.index.year
daily_future['Month']=daily_future.index.month
daily_future['CCRate']=daily_future.apply(lambda x: (0 if x['Year']<1991 else (x['Year']-1990)/80),axis=1)
daily_future

,Year,Month,CCRate
2017-10-01,2017,10,0.3375
2017-10-02,2017,10,0.3375
2017-10-03,2017,10,0.3375
2017-10-04,2017,10,0.3375
2017-10-05,2017,10,0.3375
...,...,...,...
2053-09-26,2053,9,0.7875
2053-09-27,2053,9,0.7875
2053-09-28,2053,9,0.7875
2053-09-29,2053,9,0.7875


In [117]:
start_year=1992
start_date=datetime.date(start_year, 10, 1)
start_date

datetime.date(1992, 10, 1)

In [118]:
daily_future['SampleDate']=pd.date_range(start=start_date, periods=13149, freq='D')
daily_future

,Year,Month,CCRate,SampleDate
2017-10-01,2017,10,0.3375,1992-10-01
2017-10-02,2017,10,0.3375,1992-10-02
2017-10-03,2017,10,0.3375,1992-10-03
2017-10-04,2017,10,0.3375,1992-10-04
2017-10-05,2017,10,0.3375,1992-10-05
...,...,...,...,...
2053-09-26,2053,9,0.7875,2028-09-26
2053-09-27,2053,9,0.7875,2028-09-27
2053-09-28,2053,9,0.7875,2028-09-28
2053-09-29,2053,9,0.7875,2028-09-29


In [119]:
daily_future['SampleCCRate']=daily_future.join(daily,on='SampleDate',rsuffix='_daily')['CCRate_daily']
daily_future['SampleFlow']=daily_future.join(daily,on='SampleDate',rsuffix='_daily')['Flow']

daily_future

,Year,Month,CCRate,SampleDate,SampleCCRate,SampleFlow
2017-10-01,2017,10,0.3375,1992-10-01,0.025,172.296188
2017-10-02,2017,10,0.3375,1992-10-02,0.025,170.509341
2017-10-03,2017,10,0.3375,1992-10-03,0.025,170.509341
2017-10-04,2017,10,0.3375,1992-10-04,0.025,170.509341
2017-10-05,2017,10,0.3375,1992-10-05,0.025,170.509341
...,...,...,...,...,...,...
2053-09-26,2053,9,0.7875,2028-09-26,0.000,259.369380
2053-09-27,2053,9,0.7875,2028-09-27,0.000,259.369380
2053-09-28,2053,9,0.7875,2028-09-28,0.000,254.145972
2053-09-29,2053,9,0.7875,2028-09-29,0.000,254.145972


In [121]:
daily_future['WC1_Monthly']=daily_future.join(cc_cases,on='Month')['WC1']
daily_future['WC2_Monthly']=daily_future.join(cc_cases,on='Month')['WC2']
daily_future

,Year,Month,CCRate,SampleDate,SampleCCRate,SampleFlow,WC1_Monthly,WC2_Monthly
2017-10-01,2017,10,0.3375,1992-10-01,0.025,172.296188,-0.16,-0.243920
2017-10-02,2017,10,0.3375,1992-10-02,0.025,170.509341,-0.16,-0.243920
2017-10-03,2017,10,0.3375,1992-10-03,0.025,170.509341,-0.16,-0.243920
2017-10-04,2017,10,0.3375,1992-10-04,0.025,170.509341,-0.16,-0.243920
2017-10-05,2017,10,0.3375,1992-10-05,0.025,170.509341,-0.16,-0.243920
...,...,...,...,...,...,...,...,...
2053-09-26,2053,9,0.7875,2028-09-26,0.000,259.369380,-0.14,-0.293764
2053-09-27,2053,9,0.7875,2028-09-27,0.000,259.369380,-0.14,-0.293764
2053-09-28,2053,9,0.7875,2028-09-28,0.000,254.145972,-0.14,-0.293764
2053-09-29,2053,9,0.7875,2028-09-29,0.000,254.145972,-0.14,-0.293764


In [123]:
daily_future['AdjustedCCRate']=daily_future['CCRate']-daily_future['SampleCCRate']

In [124]:
daily_future['WC1_Overall_Rate']=1+daily_future['WC1_Monthly']*daily_future['AdjustedCCRate']
daily_future['WC2_Overall_Rate']=1+daily_future['WC2_Monthly']*daily_future['AdjustedCCRate']

In [125]:
daily_future

,Year,Month,CCRate,SampleDate,SampleCCRate,SampleFlow,WC1_Monthly,WC2_Monthly,AdjustedCCRate,WC1_Overall_Rate,WC2_Overall_Rate
2017-10-01,2017,10,0.3375,1992-10-01,0.025,172.296188,-0.16,-0.243920,0.3125,0.95000,0.923775
2017-10-02,2017,10,0.3375,1992-10-02,0.025,170.509341,-0.16,-0.243920,0.3125,0.95000,0.923775
2017-10-03,2017,10,0.3375,1992-10-03,0.025,170.509341,-0.16,-0.243920,0.3125,0.95000,0.923775
2017-10-04,2017,10,0.3375,1992-10-04,0.025,170.509341,-0.16,-0.243920,0.3125,0.95000,0.923775
2017-10-05,2017,10,0.3375,1992-10-05,0.025,170.509341,-0.16,-0.243920,0.3125,0.95000,0.923775
...,...,...,...,...,...,...,...,...,...,...,...
2053-09-26,2053,9,0.7875,2028-09-26,0.000,259.369380,-0.14,-0.293764,0.7875,0.88975,0.768661
2053-09-27,2053,9,0.7875,2028-09-27,0.000,259.369380,-0.14,-0.293764,0.7875,0.88975,0.768661
2053-09-28,2053,9,0.7875,2028-09-28,0.000,254.145972,-0.14,-0.293764,0.7875,0.88975,0.768661
2053-09-29,2053,9,0.7875,2028-09-29,0.000,254.145972,-0.14,-0.293764,0.7875,0.88975,0.768661


In [126]:
daily_future['WC1_Flow']=daily_future['SampleFlow']*daily_future['WC1_Overall_Rate']
daily_future['WC2_Flow']=daily_future['SampleFlow']*daily_future['WC2_Overall_Rate']
daily_future

,Year,Month,CCRate,SampleDate,SampleCCRate,SampleFlow,WC1_Monthly,WC2_Monthly,AdjustedCCRate,WC1_Overall_Rate,WC2_Overall_Rate,WC1_Flow,WC2_Flow
2017-10-01,2017,10,0.3375,1992-10-01,0.025,172.296188,-0.16,-0.243920,0.3125,0.95000,0.923775,163.681379,159.162929
2017-10-02,2017,10,0.3375,1992-10-02,0.025,170.509341,-0.16,-0.243920,0.3125,0.95000,0.923775,161.983874,157.512284
2017-10-03,2017,10,0.3375,1992-10-03,0.025,170.509341,-0.16,-0.243920,0.3125,0.95000,0.923775,161.983874,157.512284
2017-10-04,2017,10,0.3375,1992-10-04,0.025,170.509341,-0.16,-0.243920,0.3125,0.95000,0.923775,161.983874,157.512284
2017-10-05,2017,10,0.3375,1992-10-05,0.025,170.509341,-0.16,-0.243920,0.3125,0.95000,0.923775,161.983874,157.512284
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2053-09-26,2053,9,0.7875,2028-09-26,0.000,259.369380,-0.14,-0.293764,0.7875,0.88975,0.768661,230.773906,199.367022
2053-09-27,2053,9,0.7875,2028-09-27,0.000,259.369380,-0.14,-0.293764,0.7875,0.88975,0.768661,230.773906,199.367022
2053-09-28,2053,9,0.7875,2028-09-28,0.000,254.145972,-0.14,-0.293764,0.7875,0.88975,0.768661,226.126379,195.351995
2053-09-29,2053,9,0.7875,2028-09-29,0.000,254.145972,-0.14,-0.293764,0.7875,0.88975,0.768661,226.126379,195.351995
